In [7]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from pdf_ingestor import extract_and_split_pdf
from ytb_ingestor import extract_and_split_ytb
from vectorestore import index_documents, create_vectorstore
import os 
from logger import get_logger
logger = get_logger()

In [8]:
embedding_model = HuggingFaceBgeEmbeddings(
    model_name="../../multilingual-e5-large",
    encode_kwargs={"normalize_embeddings": True},
    query_instruction="query: "
)

In [3]:
create_vectorstore(embedding_model, "../../faiss_vectorestore")

[20:34:01.207] INFO - Création d'un index FAISS vide (dim = 1024)
[20:34:01.444] INFO - Sauvegarde de l'index vide dans: ../../faiss_vectorestore
[20:34:01.447] INFO - Index vide (cosinus) créé avec succès.


In [9]:
folder_path = "../../data"
pdf_files_list = ["Renault DEU 2020.pdf", "Renault DEU 2021.pdf", "Renault URD 2022.pdf", "Renault URD 2023.pdf", "Renault 2024 Rapport d’activité.pdf"]

dict_ytb_videos_urls = {
    "PLAN STRATEGIQUE RENAULUTION" : "https://www.youtube.com/watch?app=desktop&v=EtivAvmDr2Q&t=901s",
    "Conférence résultats financiers 2021 de Renault Group" : "https://www.youtube.com/watch?v=VfIeaIFSCQA",
    "Conférence résultats financiers 2022 de Renault Group" : "https://www.youtube.com/watch?v=UWHlyjVtwT8",
    "Conférence résultats financiers 2023 de Renault Group" : "https://www.youtube.com/watch?v=B57wephix-w",
    "Conférence résultats financiers 2024 de Renault Group" : "https://www.youtube.com/watch?v=BA5ZOtWfpY0",
}

In [ ]:
pdf_chunks = []
ytb_chunks = []
CHUNK_SIZE = 450
OVERLAP = 64

for filename in (pdf_files_list) : 
    file_path = os.path.join(folder_path, filename)
    pdf_chunks.extend(extract_and_split_pdf(file_path, CHUNK_SIZE, OVERLAP))

for video_name, url in dict_ytb_videos_urls.items() : 
    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))

all_chunks = pdf_chunks + ytb_chunks

[20:34:05.642] INFO - Chargement du PDF : Renault DEU 2020.pdf
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P2' is an invalid float value
[20:38:56.027] INFO - PDF chargé avec succès : Renault DEU 2020.pdf (1 pages)
[20:38:56.310] INFO - Chunking terminé : 3423 chunks générés
[20:38:56.311] INFO - Chargement du PDF : Renault DEU 2021.pdf
[20:40:08.450] INFO - PDF chargé avec succès : Renault DEU 2021.pdf (1 pages)
[20:40:08.697] INFO - Chunking terminé : 3022 chunks générés
[20:40:08.699] INFO - Chargement du PDF : Renault URD 2022.pdf
Cannot set gray non-stroke color because /'P0' is an invalid float value
[20:40:50.802] INFO - PDF chargé avec succès : Renault URD 2022.pdf (1 pages)
[20:40:50.803] INFO - Chunking terminé : 0 chunks générés
[20:40:50.804] INFO - Chargement du PDF : Renault URD 2023.pdf
[20:42:52.168] INFO - PDF chargé avec succès : Renault URD 2023.pdf (1 pages)
[20:42:52.461] INFO - Chunking termin

In [10]:
ytb_chunks = []
CHUNK_SIZE = 450
OVERLAP = 64

for video_name, url in dict_ytb_videos_urls.items() : 
    ytb_chunks.extend(extract_and_split_ytb(url, video_name, CHUNK_SIZE, OVERLAP))

[20:51:37.973] INFO - Extraction des sous-titres pour la vidéo : PLAN STRATEGIQUE RENAULUTION (https://www.youtube.com/watch?app=desktop&v=EtivAvmDr2Q&t=901s)
[20:51:38.753] INFO - Chargement réussi : 1 documents extraits de PLAN STRATEGIQUE RENAULUTION
[20:51:38.754] ERROR - Erreur lors de l'extraction ou du chunking de la vidéo 'PLAN STRATEGIQUE RENAULUTION': create_chunks() got an unexpected keyword argument 'is_pdf'
Traceback (most recent call last):
  File "c:\Users\habou\OneDrive\Bureau\python-projects\renaulution-rag\backend\data_ingestion\ytb_ingestor.py", line 20, in extract_and_split_ytb
    all_ytb_chunks = create_chunks(documents, video_name, chunk_size, chunk_overlap)
TypeError: create_chunks() got an unexpected keyword argument 'is_pdf'
[20:51:38.755] INFO - Extraction des sous-titres pour la vidéo : Conférence résultats financiers 2021 de Renault Group (https://www.youtube.com/watch?v=VfIeaIFSCQA)
[20:51:39.463] INFO - Chargement réussi : 1 documents extraits de Conféren

In [30]:
pdf_chunks[-1]

Document(metadata={'source': 'Renault DEU 2021.pdf', 'id': '776fb501-4a34-47f3-b235-f435ebf7d76c'}, page_content='Version e-accessible par Réalisation +33 (0)1 45 58 80 00')

In [ ]:
index_documents(all_chunks, embedding_model, "../../faiss_vectorestore")

In [ ]:
pdf_chunks = extract_and_split_pdf(file_path, 450, 64)
ytb_chunks = extract_and_split_ytb("https://www.youtube.com/watch?v=VfIeaIFSCQA", 450, 64)

[13:10:08] INFO - Début de l’indexation de 247 chunks en batches de 64...
[13:10:08] INFO - Calcul des embeddings et indexation par batch avec barre de progression...
Indexing: 100%|██████████| 4/4 [01:57<00:00, 29.47s/batch]
[13:12:06] INFO - Sauvegarde de l’index local dans : ../../faiss_vectorestore
[13:12:06] INFO - Indexation terminée avec succès.
